eva.ru forum posts were downloaded using mobile forum html version (much faster). However, this forum version does not contain a year when posts were published. The code below is used to download posts using computer evar.ru forum html version to extract the full date. This is a prototype of DownloadMessagesDates.py which was run in background mode for months in several threads.

In [1]:
from bs4 import BeautifulSoup
import urllib.request
import os
import pandas as pd
import numpy as np

import time

In [9]:
Start_Message_Id=0
End_Message_Id=100
Part=200

In [10]:
Base_URL='https://eva.ru/forum/topic/message/%s.htm'
Data='/home/kate/Projects/eva/Data'

Messages_filename='missing_timestamp.csv'
Messages_full_filename=os.path.join(Data, 'Dates/', Messages_filename)

Dates_filename='Timestamp_%s.csv'%Part
Dates_full_filename=os.path.join(Data, Dates_filename)
attempts=10
delay=60

In [5]:
def GetMessageDate(Message_Id):
    html_page = urllib.request.urlopen(Base_URL%Message_Id)
    soup = BeautifulSoup(html_page)
    dates = soup.findAll('div', {'class': 'date'})
    for d in dates:
        return d.text.strip().rstrip('\n').lstrip('\n').replace('Дата: ','').replace('Время: ','')[0:14]
    else:
        return ''

In [6]:
Messages = pd.read_csv(Messages_full_filename, error_bad_lines=False, index_col=False, usecols=['Message_Id'])

In [11]:
SubsetToDownload=Messages[Start_Message_Id:End_Message_Id+1:1]

In [13]:
dates_l=list()
for index, row in SubsetToDownload.iterrows():
    print('.', end = ' '),
    cnt=0
    while cnt<attempts:
        try:
            dates_l.append(GetMessageDate(row['Message_Id']))
            cnt=attempts
        except Exception as e:
            print('Something went wrong...'+ str(e))
            cnt=cnt+1
            print('Waiting...')
            time.sleep(delay)
            print('Attempt to read message %s # %s'%(t_row['Topic_Id'],cnt+1))
    if cnt>attempts:
        print('Too many false attempts. Stop execution...')
        break
print()
print('Processing  complete')         
dates_pd=pd.DataFrame({'Message_Id': SubsetToDownload['Message_Id'].tolist(),'Timestamp': dates_l})
dates_pd.to_csv(Dates_full_filename, header=True, index=False)

. . . . . . . . . . . . . . . . . . . 
Processing  complete
